In [ ]:
%pip install mtcnn


In [21]:

import os
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from mtcnn import MTCNN
from PIL import Image
import requests
from io import BytesIO

# Load the trained model
model = load_model('C:\\Users\\pinka\\Digicrome\\DeepFake-Detector\\cnn_model.h5')  # Make sure this path is correct

# Detect and crop face using MTCNN
def detect_and_crop_face(img_path):
    if img_path.startswith('http'):
        response = requests.get(img_path)
        img = np.array(Image.open(BytesIO(response.content)).convert('RGB'))
    else:
        img = cv2.imread(img_path)

    detector = MTCNN()
    results = detector.detect_faces(img)

    if results:
        x, y, width, height = results[0]['box']
        face = img[y:y+height, x:x+width]
        face = cv2.resize(face, (128, 128))
        return face
    else:
        return cv2.resize(img, (128, 128))

# Preprocess image
def preprocess_face(face):
    img_array = image.img_to_array(face)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

# Predict real or fake
def predict_real_or_fake(img_path):
    face = detect_and_crop_face(img_path)
    if face is None:
        print(f"Could not read or process image: {img_path}")
        return
    processed_image = preprocess_face(face)
    prediction = model.predict(processed_image)
    result = 'Real' if prediction[0][0] < 0.5 else 'Fake'
    print(f"Image: {img_path} → Predicted as: {result}")

# 🔍 Example usage
img_url = r'C:\Users\pinka\Digicrome\image-7.jpg'
predict_real_or_fake(img_url)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
Image: C:\Users\pinka\Digicrome\image-7.jpg → Predicted as: Real
